# データ処理プログラミング 第3回

### Livedoorニュースコーパスで学習されたモデルをもとに、ニュース記事のカテゴリを予測する

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [ ]:
BASE_PATH = '/content/'

In [ ]:
!pip install transformers torch fugashi ipadic pandas tqdm

### テストデータの読み込み
- 読み込むファイル
    - `test.csv`  
    モデルのテストに利用するためのデータ。カラムと内容は以下の通りです。
    - label: ニュース記事のカテゴリを表すラベルを数字で表したもの。同じカテゴリーの記事は同じ数字になっています。
    - url: ニュース記事のURL
    - date: ニュース記事の日付
    - category: ニュース記事のカテゴリ
    - body: ニュース記事本文
- 読み込み後の形式  
    以下のような配列形式でデータを読み込んでください。
    なお、最終的にデータが格納される配列の変数名は`test_dataset`としてください。
    ```
    [
        {
            "label": ラベル, 
            "text": ニュース記事本文
        },
        {
            "label": ラベル, 
            "text": ニュース記事本文
        },
        ...
    ]
    ```


In [32]:
# TODO: 以下にテストデータを上記の形式で読み込むコードを書いてください
import pandas as pd
df = pd.read_csv(BASE_PATH+'test.csv')
test_dataset = []
for i in range(len(df)):
    label = df.iloc[i, 0]
    text = df.iloc[i, 4]
    test_dataset.append({
        'label': label,
        'text': text
    })

### category情報が格納されたファイルの読み込み
- 読み込むファイル
    - `categories.json`  
    ラベルのカテゴリ情報が格納されたファイル。
    以下のような形式でデータが格納されています。
    ```
    [
        {
            "label": ラベル名(数字),
            "category": カテゴリ名,
        }
        ...
    ]
    
    ```
- 読み込み後の形式  
    そのままのPython dict型の要素が格納された配列で読み込んでください。
    配列が格納される変数名は`categories`としてください。

In [33]:
# TODO: 以下にcategory情報を上記の形式で読み込むコードを書いてください
import json
with open(BASE_PATH+'categories.json') as f:
    categories = json.load(f)

### データの前処理

In [34]:
# seed値の設定
from transformers import set_seed
seed = 42
set_seed(seed)

In [28]:
# TODO: トークナイザを読み込むコードを書いて下さい。
# トークナイザは「'cl-tohoku/bert-base-japanese-whole-word-masking'」用のモデルを利用して下さい。この名称+tokenizer等で検索するか、学習スクリプトを参考にしてみて下さい。
# なお、読み込んだtokenizerはtokenzierという変数に格納して下さい。
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


In [30]:
model_max_length = tokenizer.model_max_length

In [35]:
# TODO: test_datasetの各辞書型のデータをfor文で回し、辞書型のデータ内の'text'キーの値をトークナイザに入力し、その結果を元の辞書型のデータに'encoding'キーで追加してください。
# tokenizerに渡すパラメーターは以下のものを利用して下さい。
# padding='max_length', truncation=True, max_length=model_max_length, return_tensors='pt'
for data in test_dataset:
    data['encoding'] = tokenizer(data['text'], padding='max_length', truncation=True, max_length=model_max_length, return_tensors='pt')

In [36]:

# データセットの確認
print(test_dataset[0])

{'label': 3, 'text': 'ソニープロフェッショナルセミナー〜3Dの世界【ビデオSALON】【開催概要】◎ソニープロフェッショナルセミナー\u3000〜複眼カムコーダーとVEGASで3Dの世界へ〜 開催-----------------------------------------------------------------------3Dカムコーダーの登場でより簡単に3D撮影ができるようになったとはいえ、3Dの基礎を知っているのとそうでないのとでは撮影映像の質も違ってくるはず。3Dに興味のある方、実際にこれから3D制作に携わっていく方を対象とした入門編。◇日時：2012年3月7日(水) 13:00〜15:00◇会場：ソニー株式会社 本社（JR品川駅港南口）◇入場：無料（定員50名）\u3000※事前申込が必要です。申込締切3/6(火)\u3000※定員になり次第、締め切らせていただきます。▼お申し込みはこちらhttp://www.sony.jp/professional/event/info/pb20120307.html■ビデオSALON 最新記事・ニコニコ動画、生中継用エンコーダーや画質向上など新サービスを続々発表・ふるいちやすし氏の新作上映とトークイベントをアップルストアで開催・ブラックマジックデザイン、Thunderbolt対応のIntensity Shuttleを発売・TVF2012の入賞30作品が決定■ビデオSALON最新号    ■関連記事・寝つけない夜に寝室で愛用…滝クリもオススメのスマートAVライフはパナソニックでチェック【話題】・ついにケーブル不要？ 無線での高速データ送受信実現か\u3000東京工業大学とソニーが世界最高速のミリ波無線用LSIを共同開発【ビデオSALON】・東電リストラ！\u3000クビを切られたのは人気キャラクター「でんこちゃん」だった【話題】・家庭菜園の流行で注目！\u3000初心者や女性にもぴったりな軽くて使いやすい「ミニ耕運機\u3000ＬＧＣ１２０」【売れ筋チェック】・「最新」とつぶやくだけで最新ニュースをゲット！\u3000定期配信も可能な便利機能が「LINE」に登場【話題】', 'encoding': {'input_ids': tensor([[    2,  6369,  2496,  

### モデルの読み込み

In [37]:
# 環境の設定
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [38]:
# モデルの読み込み
from transformers import BertForSequenceClassification
import torch
model_path = BASE_PATH+'model/model.pth'
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=len(categories))
# 学習済みモデルの重みを読み込む
model.load_state_dict(torch.load(model_path))
model.eval() # モデルを評価モードにする
model.to(device) # モデルをGPUに載せる

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [39]:
# テストデータのDataLoaderを作成
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [47]:
# 評価用のスクリプト
from tqdm import tqdm
texts = []
pred_labels = []
true_labels = []
for data in tqdm(test_dataloader):
    label = data['label']
    encoding = data['encoding']
    output = model(**{k: v.squeeze(0).to(device) for k, v in encoding.items()})
    
    pred_label = torch.argmax(output.logits, dim=1)
    pred_labels.append(pred_label.item())
    true_labels.append(label)
    texts.append(data['text'])
assert len(pred_labels) == len(true_labels) 

100%|██████████| 738/738 [00:47<00:00, 15.48it/s] 


In [32]:
accuracy = (torch.tensor(pred_labels) == torch.tensor(true_labels)).float().sum().item() / len(true_labels)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9525745257452575


In [49]:
# 実際に予測されたラベルと正解ラベルを表示
for i in range(5):
    print(f"Text: {texts[i]}")
    print(f"model output: {categories[pred_labels[i]]}")
    print(f"True label: {categories[true_labels[i]]}")
    print()

Text: ['ソニープロフェッショナルセミナー〜3Dの世界【ビデオSALON】【開催概要】◎ソニープロフェッショナルセミナー\u3000〜複眼カムコーダーとVEGASで3Dの世界へ〜 開催-----------------------------------------------------------------------3Dカムコーダーの登場でより簡単に3D撮影ができるようになったとはいえ、3Dの基礎を知っているのとそうでないのとでは撮影映像の質も違ってくるはず。3Dに興味のある方、実際にこれから3D制作に携わっていく方を対象とした入門編。◇日時：2012年3月7日(水) 13:00〜15:00◇会場：ソニー株式会社 本社（JR品川駅港南口）◇入場：無料（定員50名）\u3000※事前申込が必要です。申込締切3/6(火)\u3000※定員になり次第、締め切らせていただきます。▼お申し込みはこちらhttp://www.sony.jp/professional/event/info/pb20120307.html■ビデオSALON 最新記事・ニコニコ動画、生中継用エンコーダーや画質向上など新サービスを続々発表・ふるいちやすし氏の新作上映とトークイベントをアップルストアで開催・ブラックマジックデザイン、Thunderbolt対応のIntensity Shuttleを発売・TVF2012の入賞30作品が決定■ビデオSALON最新号    ■関連記事・寝つけない夜に寝室で愛用…滝クリもオススメのスマートAVライフはパナソニックでチェック【話題】・ついにケーブル不要？ 無線での高速データ送受信実現か\u3000東京工業大学とソニーが世界最高速のミリ波無線用LSIを共同開発【ビデオSALON】・東電リストラ！\u3000クビを切られたのは人気キャラクター「でんこちゃん」だった【話題】・家庭菜園の流行で注目！\u3000初心者や女性にもぴったりな軽くて使いやすい「ミニ耕運機\u3000ＬＧＣ１２０」【売れ筋チェック】・「最新」とつぶやくだけで最新ニュースをゲット！\u3000定期配信も可能な便利機能が「LINE」に登場【話題】']
model output: {'label': 3, 'category': 'kaden-channel'}
True label: {'l

### GPT-2(デコーダーモデル)を触ってみる

In [36]:
!pip install sentencepiece

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

gpt_tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium", use_fast=False)
gpt_tokenizer.do_lower_case = True  

decoder_model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")
decoder_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=32000, bias=False)
)

In [24]:
# 文章生成時のパラメータ
# TODO: パラメーターの値を変えたり追加したりして、モデルの出力がどのように変化するかをみてみましょう。各種パラメーターがどのような働きなのかを理解しながら変えてみるのが良いかもしれません。
params = {
    "max_length":100,
    "top_k" : 50,
    "temperature": 0.7,
    "do_sample": True,
    # "num_beams":5,
    # "early_stopping":True,
    # "no_repeat_ngram_size":2,
}


In [25]:
# NOTE: start_textを色々変えてみると...?????
start_text = "昔々あるところに、"
with torch.no_grad():
    input_ids = gpt_tokenizer.encode(start_text, add_special_tokens=False, return_tensors="pt")
    output_ids = decoder_model.generate(input_ids.to(device), pad_token_id=gpt_tokenizer.pad_token_id, **params)

print(gpt_tokenizer.decode(output_ids.tolist()[0]))

昔々あるところに、おじいさんとおばあさんが住んでいました。 おじいさんはいつもお店番をしていて、おばあさんはおじいさんの家に遊びに行っていました。 でもおばあさんは、いつもおじいさんと一緒にいるので、だんだんとおじいさんのことが嫌いになっていきました。 おじいさんは、おじいさんの家に遊びに行ったとき、何だかお腹
